In [1]:
# use if autocompletion is not working
%config Completer.use_jedi = False

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import pandas as pd
import json
from tqdm import tqdm

from typing import List
import re
import numpy as np
from datetime import date
from copy import deepcopy

In [4]:
from modern_slavery_registry import get_root_path

In [5]:
PROJECT_PATH = get_root_path()
DATA_PATH = os.path.join(PROJECT_PATH, "data")
SHEETS_PATH = os.path.join(PROJECT_PATH, "data", "sheets")

In [6]:
os.listdir(SHEETS_PATH)

['.ipynb_checkpoints',
 'Continents_to_CountryNames.csv',
 'fuzzy_matches.xlsx',
 'combined_modern_slavery_statements.xlsx',
 'modern_slavery_dataset.csv',
 'modern_slavery_dataset_nitin.xlsx',
 'subset_data.xlsx',
 'CountryCodes_to_CountryNames.csv',
 'subset_data_v3.xlsx',
 'subset_data_v2.xlsx',
 'ext_shawn_creds.csv']

## Data - AWS

In [7]:
aws_df = pd.read_csv(os.path.join(SHEETS_PATH, "modern_slavery_dataset.csv"))
aws_df.head()

,Company ID,Company,Is Publisher,Statement ID,URL,Override URL,Companies House Number,Industry,HQ,Is Also Covered,UK Modern Slavery Act,California Transparency in Supply Chains Act,Australia Modern Slavery Act,Period Covered,Text
0,7676,"""K"" Line Holding Europe Limited",True,35092.0,https://img1.wsimg.com/blobby/go/7695baff-3f0f...,NaN,05005018,Marine,United Kingdom,False,True,False,False,2018-2019,K LINE\nHOLDINC (EUROPE) LTD.\nModern Slavery ...
1,28660,"""K"" Line Bulk Shipping (UK) Limited",False,35092.0,https://img1.wsimg.com/blobby/go/7695baff-3f0f...,NaN,04830352,Marine,United Kingdom,True,True,False,False,2018-2019,K LINE\nHOLDINC (EUROPE) LTD.\nModern Slavery ...
2,28659,"""K"" Line (Europe) Limited",False,35092.0,https://img1.wsimg.com/blobby/go/7695baff-3f0f...,NaN,05639474,Marine,United Kingdom,True,True,False,False,2018-2019,K LINE\nHOLDINC (EUROPE) LTD.\nModern Slavery ...
3,28661,"""K"" Line LNG Shipping Limited",False,35092.0,https://img1.wsimg.com/blobby/go/7695baff-3f0f...,NaN,NaN,Marine,United Kingdom,True,True,False,False,2018-2019,K LINE\nHOLDINC (EUROPE) LTD.\nModern Slavery ...
4,28658,Polar LNG Shipping (UK) Limited,False,35092.0,https://img1.wsimg.com/blobby/go/7695baff-3f0f...,NaN,02205323,Marine,United Kingdom,True,True,False,False,2018-2019,K LINE\nHOLDINC (EUROPE) LTD.\nModern Slavery ...


## Data - ours

Same columns as in AWS data

In [8]:
# Picking last year for every statement
our_df_unclean = pd.read_excel(
    os.path.join(SHEETS_PATH, "modern_slavery_dataset_nitin.xlsx"))

years_fixed = []
for year in our_df_unclean["Period Covered"]:
    if year is None:
        years_fixed.append(None)
        continue
    else:
        year = np.sort(str(year).split("-"))
        years_fixed.append(year[-1])
our_df_unclean["Period Covered"] = years_fixed
del years_fixed

our_df_unclean.head()

,Company ID,Company,Is Publisher,Statement ID,URL,Override URL,Companies House Number,Industry,HQ,Is Also Covered,UK Modern Slavery Act,California Transparency in Supply Chains Act,Australia Modern Slavery Act,Period Covered,statement
0,7676,"""K"" Line Holding Europe Limited",True,35092,https://img1.wsimg.com/blobby/go/7695baff-3f0f...,NaN,05005018,Marine,United Kingdom,False,True,False,False,2019,66 99 “K” Line Holding (Europe) Limited kM K L...
1,28660,"""K"" Line Bulk Shipping (UK) Limited",False,35092,https://img1.wsimg.com/blobby/go/7695baff-3f0f...,NaN,04830352,Marine,United Kingdom,True,True,False,False,2019,66 99 “K” Line Holding (Europe) Limited kM K L...
2,28659,"""K"" Line (Europe) Limited",False,35092,https://img1.wsimg.com/blobby/go/7695baff-3f0f...,NaN,05639474,Marine,United Kingdom,True,True,False,False,2019,66 99 “K” Line Holding (Europe) Limited kM K L...
3,28661,"""K"" Line LNG Shipping Limited",False,35092,https://img1.wsimg.com/blobby/go/7695baff-3f0f...,NaN,NaN,Marine,United Kingdom,True,True,False,False,2019,66 99 “K” Line Holding (Europe) Limited kM K L...
4,28658,Polar LNG Shipping (UK) Limited,False,35092,https://img1.wsimg.com/blobby/go/7695baff-3f0f...,NaN,02205323,Marine,United Kingdom,True,True,False,False,2019,66 99 “K” Line Holding (Europe) Limited kM K L...


### Data(cleaned version and with additional information) - ours

In [9]:
our_df_clean = pd.read_json(os.path.join(DATA_PATH, "subset-data-with-additional-info-v3.json"))
our_df_clean.drop(["years_with_and_without_act", "years_with_act", "len(final_statement_cleaned)"], axis=1, inplace=True)
our_df_clean.head()

,URL,Company,final_statement,final_statement_cleaned,additional_info
0,https://1spatial.com/who-we-are/legal/modern-s...,1Spatial Plc,1Spatial Modern Slavery Act Policy Statement H...,home solution government boundary law enforcem...,"{'id': ['903081'], 'headquarters': ['GB'], 'se..."
1,https://www.shazans.com/slavery-and-human-traf...,1Stop Halal Limited,Slavery and Human Trafficking Statement – Shaz...,shazans shazan food continue monitor covid 19 ...,"{'id': ['903810'], 'headquarters': ['GB'], 'se..."
2,https://www.business-humanrights.org/sites/def...,1st Step Solutions Limited,7/28/2019 Modern Slavery Statement 2018 - 1st ...,28 2019 2018 statement make pursuant sec 54 20...,"{'id': ['900634'], 'headquarters': ['GB'], 'se..."
3,https://www.2agriculture.com/wp-content/upload...,2 Agriculture Limited,fh Modern Slavery Act 2015: slavery and human ...,fh 2015 introduction uk act require business s...,"{'id': ['901400'], 'headquarters': ['GB'], 'se..."
4,https://www.2agriculture.com/wp-content/upload...,2 Agriculture Limited,1 Modern Slavery Act 2015: slavery and human t...,2015 introduction uk act require business stat...,"{'id': ['901400'], 'headquarters': ['GB'], 'se..."


In [10]:
## combining uncleaned/aws and cleaned dfs 
cols_to_keep = [
    "statement",
    "Company", 
    "Industry",
    "HQ",
    "UK Modern Slavery Act",
    "California Transparency in Supply Chains Act",
    "Australia Modern Slavery Act",
    "Period Covered"]

our_df_unclean = our_df_unclean[cols_to_keep]
our_df_unclean.columns = ["FS_" + col.lower().replace(" ", "_") for col in our_df_unclean.columns]

final_df = pd.merge(
    our_df_clean[["final_statement", "Company", "final_statement_cleaned"]],
    our_df_unclean[~our_df_unclean["FS_statement"].isna()],
    left_on=["final_statement", "Company"],
    right_on=["FS_statement", "FS_company"], 
    how="left").drop_duplicates()
final_df.drop(["FS_statement", "FS_company"], axis=1, inplace=True)
final_df.head()

,final_statement,Company,final_statement_cleaned,FS_industry,FS_hq,FS_uk_modern_slavery_act,FS_california_transparency_in_supply_chains_act,FS_australia_modern_slavery_act,FS_period_covered
0,1Spatial Modern Slavery Act Policy Statement H...,1Spatial Plc,home solution government boundary law enforcem...,Software,United Kingdom,True,False,False,2017
1,Slavery and Human Trafficking Statement – Shaz...,1Stop Halal Limited,shazans shazan food continue monitor covid 19 ...,Food Products,United Kingdom,True,False,False,2018
2,7/28/2019 Modern Slavery Statement 2018 - 1st ...,1st Step Solutions Limited,28 2019 2018 statement make pursuant sec 54 20...,Professional Services,United Kingdom,True,False,False,2018
3,fh Modern Slavery Act 2015: slavery and human ...,2 Agriculture Limited,fh 2015 introduction uk act require business s...,Food Products,United Kingdom,True,False,False,2018
4,1 Modern Slavery Act 2015: slavery and human t...,2 Agriculture Limited,2015 introduction uk act require business stat...,Food Products,United Kingdom,True,False,False,2017


In [11]:
final_df["FS_period_covered"].value_counts(dropna=False)

2018    3079
2016    2777
2019    2204
2017    1761
NaN      693
nan      404
2020     309
2015      23
2014      10
2021       5
Name: FS_period_covered, dtype: int64

In [13]:
# bringing data at statement level
final_dict_1 = {}
for stat in tqdm(final_df["final_statement_cleaned"].unique(), leave=False, position=0):
    rows = final_df[final_df["final_statement_cleaned"]==stat].to_dict("list")
    
    n = len(rows["final_statement"])
    keys = list(rows.keys())
    keys.remove("final_statement_cleaned")
#     keys.remove("final_statement")
    
    processed_rows = []
    for i in range(n):
        processed_row_i = {}
        for k in keys: 
            processed_row_i[k] = rows[k][i] 
        
        processed_rows.append(processed_row_i)
        
    final_dict_1[stat] = processed_rows
    
# dropping duplicate entries across any statement
final_dict_clean_1 = {}
for k, v in tqdm(final_dict_1.items(), leave=False, position=0):
    if len(v) < 1:
        continue
    else:
        v = pd.DataFrame.from_dict(v).drop_duplicates().to_dict("records")
    final_dict_clean_1[k] = v
    
del final_dict_1

In [14]:
# checking if a statement has 2 entries
for k, v in final_dict_clean_1.items():
    if len(v) > 1:
        print(*final_dict_clean_1[k], sep="\n")
        break

{'final_statement': "Modern Slavery Statement | A B Graphic International Your browser is unsupported and may have security vulnerabilities! Upgrade to a newer browser to experience this site in all it's glory. Skip to main content About Us Distributors Products Contact Us News English German Spanish French Italian Modern Slavery Statement Introduction from Anthony Gardner, Chief Executive Officer We are committed to improving our practices to combat slavery and human trafficking. This statement is made pursuant to section 54(1) of the Modern Slavery Act 2015 and constitutes the A B Graphic International Group's slavery and human trafficking statement for the financial year ending 30 April 2018 Organisation's Structure We are a provider of a range of Label Converting Machinery principally in the Label and Packaging industry. Our ultimate parent company is GHB Holding Limited whose head office is in England. The Group has in excess of 350 employees worldwide and operate in England, Amer

### Extracting years explicitly

In [15]:
our_df_clean.head()

,URL,Company,final_statement,final_statement_cleaned,additional_info
0,https://1spatial.com/who-we-are/legal/modern-s...,1Spatial Plc,1Spatial Modern Slavery Act Policy Statement H...,home solution government boundary law enforcem...,"{'id': ['903081'], 'headquarters': ['GB'], 'se..."
1,https://www.shazans.com/slavery-and-human-traf...,1Stop Halal Limited,Slavery and Human Trafficking Statement – Shaz...,shazans shazan food continue monitor covid 19 ...,"{'id': ['903810'], 'headquarters': ['GB'], 'se..."
2,https://www.business-humanrights.org/sites/def...,1st Step Solutions Limited,7/28/2019 Modern Slavery Statement 2018 - 1st ...,28 2019 2018 statement make pursuant sec 54 20...,"{'id': ['900634'], 'headquarters': ['GB'], 'se..."
3,https://www.2agriculture.com/wp-content/upload...,2 Agriculture Limited,fh Modern Slavery Act 2015: slavery and human ...,fh 2015 introduction uk act require business s...,"{'id': ['901400'], 'headquarters': ['GB'], 'se..."
4,https://www.2agriculture.com/wp-content/upload...,2 Agriculture Limited,1 Modern Slavery Act 2015: slavery and human t...,2015 introduction uk act require business stat...,"{'id': ['901400'], 'headquarters': ['GB'], 'se..."


In [16]:
def extract_year(text: str) -> List[str]:
    """Extract years present in text."""
    years = set() 
    formats = [
        r"\b[0-9]{2}/[0-9]{2}/[1-2][0-9]\b", 
        r"\b[0-9]{2}/[0-9]{2}/[2][0][1][0-9]\b",
        r"\b[2][0][1][0-9]\b"]
    for f in formats:         
        for year in re.findall(f, text):
            if "/" in year:
                year = year.split("/")[-1]
                if len(year) < 4:
                    year = "20" + year 
            years.add(year)
            
    years = [year for year in years if int(year) <= date.today().year]
    return np.sort(list(years))

In [17]:
def find_statement_year(statement: str):
    """Find year from modern slavery statement."""
    statement = statement.lower()
    ignore_phrases = [
        r"modern slavery act [0-9]{4}", 
        r"transparency in supply chains act [0-9]{4}", 
        r"duty of diligence [0-9]{4}"]

    for phrase in ignore_phrases:
        for s in re.findall(phrase, statement):
            statement = statement.replace(s, " ")
    
    statement =  " ".join(statement.split())
    
    years = extract_year(statement)
    
    if len(years) < 1:
        return None
    
    return years[-1]

In [18]:
# extracting year from statements
our_df_clean["year"] = [find_statement_year(s) for s in tqdm(our_df_clean["final_statement"], leave=False, position=0)]

In [19]:
our_df_clean.head()

,URL,Company,final_statement,final_statement_cleaned,additional_info,year
0,https://1spatial.com/who-we-are/legal/modern-s...,1Spatial Plc,1Spatial Modern Slavery Act Policy Statement H...,home solution government boundary law enforcem...,"{'id': ['903081'], 'headquarters': ['GB'], 'se...",None
1,https://www.shazans.com/slavery-and-human-traf...,1Stop Halal Limited,Slavery and Human Trafficking Statement – Shaz...,shazans shazan food continue monitor covid 19 ...,"{'id': ['903810'], 'headquarters': ['GB'], 'se...",2019
2,https://www.business-humanrights.org/sites/def...,1st Step Solutions Limited,7/28/2019 Modern Slavery Statement 2018 - 1st ...,28 2019 2018 statement make pursuant sec 54 20...,"{'id': ['900634'], 'headquarters': ['GB'], 'se...",2019
3,https://www.2agriculture.com/wp-content/upload...,2 Agriculture Limited,fh Modern Slavery Act 2015: slavery and human ...,fh 2015 introduction uk act require business s...,"{'id': ['901400'], 'headquarters': ['GB'], 'se...",2019
4,https://www.2agriculture.com/wp-content/upload...,2 Agriculture Limited,1 Modern Slavery Act 2015: slavery and human t...,2015 introduction uk act require business stat...,"{'id': ['901400'], 'headquarters': ['GB'], 'se...",2018


In [20]:
our_df_clean["year"].value_counts(dropna=False)

2019    5033
2018    1686
NaN     1073
2017    1070
2016     421
2010     246
2015     193
2012     159
2014      44
2011      26
2020      21
2013      20
2021       1
Name: year, dtype: int64

In [21]:
# bringing cleaned version data at statement level
final_dict_2 = {}
for stat in tqdm(our_df_clean["final_statement_cleaned"], leave=False, position=0):
    rows = deepcopy(our_df_clean[our_df_clean["final_statement_cleaned"]==stat])
    rows.drop(["final_statement", "final_statement_cleaned"], axis=1, inplace=True)
    rows = rows.to_dict("records")
    
    cols_to_keep = ["Company", "year", "headquarters", "sectors"]
    processed_rows = []
    for row in rows:
        row = deepcopy(row)
        info = deepcopy(row["additional_info"])

        for k,v in info.items():
            row[k] = v
        
        keys = deepcopy(list(row.keys()))
        for k in keys:
            if k not in cols_to_keep:
                del row[k]
        
        processed_rows.append(row)
        
    final_dict_2[stat] = processed_rows
    
# merging information
final_dict_clean_2 = {}
for k in tqdm(final_dict_clean_1.keys(), leave=False, position=0):
    v1 = pd.DataFrame.from_dict(final_dict_clean_1[k])
    v2 = pd.DataFrame.from_dict(final_dict_2[k])
    final_dict_clean_2[k] = pd.merge(v1, v2, on="Company", how="left").to_dict("records")
del final_dict_clean_1, final_dict_2

In [22]:
# checking if a statement has 2 entries
for k, v in final_dict_clean_2.items():
    if len(v) > 1:
        print(*final_dict_clean_2[k], sep="\n")
        break

{'final_statement': "Modern Slavery Statement | A B Graphic International Your browser is unsupported and may have security vulnerabilities! Upgrade to a newer browser to experience this site in all it's glory. Skip to main content About Us Distributors Products Contact Us News English German Spanish French Italian Modern Slavery Statement Introduction from Anthony Gardner, Chief Executive Officer We are committed to improving our practices to combat slavery and human trafficking. This statement is made pursuant to section 54(1) of the Modern Slavery Act 2015 and constitutes the A B Graphic International Group's slavery and human trafficking statement for the financial year ending 30 April 2018 Organisation's Structure We are a provider of a range of Label Converting Machinery principally in the Label and Packaging industry. Our ultimate parent company is GHB Holding Limited whose head office is in England. The Group has in excess of 350 employees worldwide and operate in England, Amer

In [23]:
final_dict_clean_2[list(final_dict_clean_2.keys())[0]]

[{'final_statement': '1Spatial Modern Slavery Act Policy Statement Home Solutions Solutions Government Boundaries Law Enforcement Rural Payments Next Generation 911 Utilities ArcGIS Utility Network Accelerator Traffic Management Plan Automation (TMPA) Automated Schematics Pipe Inference Incident Management Leakage Solutions Transport and Infrastructure Supply Chain Data Quality Highway Performance Monitoring System Next Generation 911 Services Consultancy Support Training 1Spatial Training FME Training Geocortex Training "1Spatial provides excellent knowledge and support to meet our current business needs. Individuals also go the extra mile in responding to critical issues and timescales, which is always very much appreciated and essential to the continued success of our business." Danny O\'Reilly | DAERA Industries Industries Utilities Water and Wastewater Gas Electricity Telecommunications Government National Mapping Agencies Land and Environmental Management Public Sector Procuremen

In [24]:
with open(os.path.join(DATA_PATH, 'subset-data-with-additional-info-v5.json'), 'w') as f:
    json.dump(final_dict_clean_2, f)

In [25]:
with open(os.path.join(DATA_PATH, 'subset-data-with-additional-info-v5.json')) as fp:
    json_obj = json.load(fp)

In [26]:
json_obj[list(json_obj.keys())[0]]

[{'final_statement': '1Spatial Modern Slavery Act Policy Statement Home Solutions Solutions Government Boundaries Law Enforcement Rural Payments Next Generation 911 Utilities ArcGIS Utility Network Accelerator Traffic Management Plan Automation (TMPA) Automated Schematics Pipe Inference Incident Management Leakage Solutions Transport and Infrastructure Supply Chain Data Quality Highway Performance Monitoring System Next Generation 911 Services Consultancy Support Training 1Spatial Training FME Training Geocortex Training "1Spatial provides excellent knowledge and support to meet our current business needs. Individuals also go the extra mile in responding to critical issues and timescales, which is always very much appreciated and essential to the continued success of our business." Danny O\'Reilly | DAERA Industries Industries Utilities Water and Wastewater Gas Electricity Telecommunications Government National Mapping Agencies Land and Environmental Management Public Sector Procuremen

In [27]:
# Fixing headquaters
distinct_hq = set()
for k, list_dict in tqdm(json_obj.items(), leave=False, position=0):
    for dict_ in list_dict:
        for hq in dict_["headquarters"]:
            distinct_hq.add(hq)
            
json_obj_flat = {}
for k, list_dict in tqdm(json_obj.items(), leave=False, position=0):
    new_list_dict = []
    for dict_ in list_dict:
        new_dict_ = deepcopy(dict_)
        del new_dict_["headquarters"]
        
        for hq in distinct_hq:
            new_dict_[f"HQ_{hq}"] = 0

        for hq in dict_["headquarters"]:
            new_dict_[f"HQ_{hq}"] = 1
        new_list_dict.append(new_dict_)
    json_obj_flat[k] = new_list_dict

In [28]:
json_obj_flat[list(json_obj_flat.keys())[0]]

[{'final_statement': '1Spatial Modern Slavery Act Policy Statement Home Solutions Solutions Government Boundaries Law Enforcement Rural Payments Next Generation 911 Utilities ArcGIS Utility Network Accelerator Traffic Management Plan Automation (TMPA) Automated Schematics Pipe Inference Incident Management Leakage Solutions Transport and Infrastructure Supply Chain Data Quality Highway Performance Monitoring System Next Generation 911 Services Consultancy Support Training 1Spatial Training FME Training Geocortex Training "1Spatial provides excellent knowledge and support to meet our current business needs. Individuals also go the extra mile in responding to critical issues and timescales, which is always very much appreciated and essential to the continued success of our business." Danny O\'Reilly | DAERA Industries Industries Utilities Water and Wastewater Gas Electricity Telecommunications Government National Mapping Agencies Land and Environmental Management Public Sector Procuremen

In [29]:
# Fixing sectors
distinct_sectors = set()
for k, list_dict in tqdm(json_obj.items(), leave=False, position=0):
    for dict_ in list_dict:
        for sector in dict_["sectors"].keys():
            distinct_sectors.add(sector)
            
for k, list_dict in tqdm(json_obj_flat.items(), leave=False, position=0):
    new_list_dict = []
    for dict_ in list_dict:
        new_dict_ = deepcopy(dict_)
        
        for sector in distinct_sectors:
            new_dict_[f"SECTOR_{sector}"] = 0

        for sector in dict_["sectors"].keys():
            new_dict_[f"SECTOR_{sector}"] = 1
        new_list_dict.append(new_dict_)
    json_obj_flat[k] = new_list_dict

In [30]:
json_obj_flat[list(json_obj_flat.keys())[0]]

[{'final_statement': '1Spatial Modern Slavery Act Policy Statement Home Solutions Solutions Government Boundaries Law Enforcement Rural Payments Next Generation 911 Utilities ArcGIS Utility Network Accelerator Traffic Management Plan Automation (TMPA) Automated Schematics Pipe Inference Incident Management Leakage Solutions Transport and Infrastructure Supply Chain Data Quality Highway Performance Monitoring System Next Generation 911 Services Consultancy Support Training 1Spatial Training FME Training Geocortex Training "1Spatial provides excellent knowledge and support to meet our current business needs. Individuals also go the extra mile in responding to critical issues and timescales, which is always very much appreciated and essential to the continued success of our business." Danny O\'Reilly | DAERA Industries Industries Utilities Water and Wastewater Gas Electricity Telecommunications Government National Mapping Agencies Land and Environmental Management Public Sector Procuremen

In [31]:
# Fixing subsectors
distinct_subsectors = set()
for k, list_dict in tqdm(json_obj.items(), leave=False, position=0):
    for dict_ in list_dict:
        for subsectors in dict_["sectors"].values():
            for subsector in subsectors:
                distinct_subsectors.add(subsector)
                
for k, list_dict in tqdm(json_obj_flat.items(), leave=False, position=0):
    new_list_dict = []
    for dict_ in list_dict:
        new_dict_ = deepcopy(dict_)
        
        for subsector in distinct_subsectors:
            new_dict_[f"SUBSECTOR_{subsector}"] = 0
        
        for subsectors in dict_["sectors"].values():
            for subsector in subsectors:
                new_dict_[f"SUBSECTOR_{subsector}"] = 1
                
        new_list_dict.append(new_dict_)
    json_obj_flat[k] = new_list_dict

In [32]:
json_obj_flat[list(json_obj_flat.keys())[0]]

[{'final_statement': '1Spatial Modern Slavery Act Policy Statement Home Solutions Solutions Government Boundaries Law Enforcement Rural Payments Next Generation 911 Utilities ArcGIS Utility Network Accelerator Traffic Management Plan Automation (TMPA) Automated Schematics Pipe Inference Incident Management Leakage Solutions Transport and Infrastructure Supply Chain Data Quality Highway Performance Monitoring System Next Generation 911 Services Consultancy Support Training 1Spatial Training FME Training Geocortex Training "1Spatial provides excellent knowledge and support to meet our current business needs. Individuals also go the extra mile in responding to critical issues and timescales, which is always very much appreciated and essential to the continued success of our business." Danny O\'Reilly | DAERA Industries Industries Utilities Water and Wastewater Gas Electricity Telecommunications Government National Mapping Agencies Land and Environmental Management Public Sector Procuremen

In [33]:
# final flattening operation
final_json_obj_flat = []
for k, v1 in tqdm(json_obj_flat.items(), leave=False, position=0):
    new_flat_list = []
    for v2 in v1:
        new_dict_ = deepcopy(v2)
        del new_dict_["sectors"]
        new_dict_["statement"] = k
        new_flat_list.append(new_dict_)
    final_json_obj_flat += new_flat_list

In [34]:
final_flat_df = pd.DataFrame.from_dict(final_json_obj_flat)

same_hqs = {
    "ES": "Spain", 
    "LU": "Luxembourg", 
    "NL": "Netherlands",
    "HK": "Hong Kong",
    "CA": "Canada",
    "TW": "Taiwan", 
    "SE": "Sweden", 
    "AE": "United Arab Emirates", 
    "US": "United States",
    "JP": "Japan", 
    "GB": "United Kingdom",
    "FI": "Finland", 
    "AU": "Australia",
    "DE": "Germany",
    "IE": "Ireland", 
    "AT": "Austria",
    "BE": "Belgium", 
    "IN": "India"}
for hq1, hq2 in same_hqs.items():
    final_flat_df[f"HQ_{hq1}"] = np.where((final_flat_df[f"HQ_{hq1}"]==1) | (final_flat_df[f"HQ_{hq2}"]==1),1,0)
    final_flat_df.drop(f"HQ_{hq2}", inplace=True, axis=1)

In [35]:
print(*final_flat_df.columns, sep=", ")

final_statement, Company, FS_industry, FS_hq, FS_uk_modern_slavery_act, FS_california_transparency_in_supply_chains_act, FS_australia_modern_slavery_act, FS_period_covered, year, HQ_KZ, HQ_RU, HQ_CY, HQ_US, HQ_GG, HQ_KR, HQ_BE, HQ_VN, HQ_GB, HQ_LU, HQ_SA, HQ_HN, HQ_CH, HQ_ZA, HQ_CA, HQ_QA, HQ_OM, HQ_NZ, HQ_AF, HQ_IL, HQ_IT, HQ_SG, HQ_AE, HQ_TH, HQ_JP, HQ_NL, HQ_DE, HQ_GL, HQ_AT, HQ_TW, HQ_ES, HQ_MY, HQ_KW, HQ_CN, HQ_FI, HQ_None, HQ_NO, HQ_PH, HQ_AU, HQ_BM, HQ_MX, HQ_BR, HQ_GE, HQ_CL, HQ_PL, HQ_IE, HQ_HK, HQ_UG, HQ_TR, HQ_FR, HQ_DK, HQ_SE, HQ_IN, HQ_IO, HQ_JE, HQ_BH, SECTOR_Charity/Non-Profit, SECTOR_Shipping & handling, SECTOR_Household Products, SECTOR_Other, SECTOR_Insurance, SECTOR_Construction & Engineering, SECTOR_Air Freight & Logistics, SECTOR_Safety/environment, SECTOR_Public Entities, SECTOR_Garden/landscaping, SECTOR_Equity Real Estate Investment Trusts (REITs), SECTOR_Road & Rail, SECTOR_Consumer products/retail, SECTOR_Beverages, SECTOR_Independent Power and Renewable Elect

In [36]:
final_flat_df.fillna(value="#NA", inplace=True)

In [37]:
final_flat_df.to_excel(os.path.join(SHEETS_PATH, "subset_data_v4.xlsx"), index=False)